In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cnpj_final = pd.read_csv('/content/drive/MyDrive/TCC - PUC/cnpj_final.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0)

In [ ]:
municipios_siafi = pd.read_csv('//content/drive/MyDrive/TCC - PUC/TABELA DE MUNICÍPIOS DO SIAFI.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0,decimal=',')

In [ ]:
municipios_siafi.columns = ['codigo_municipio_siafi', 'cnpj_base', 'descricao_municipio', 'uf','codigo_ibge']

In [ ]:
municipios_siafi

,codigo_municipio_siafi,cnpj_base,descricao_municipio,uf,codigo_ibge
0,1,"5,89363E+12",GUAJARA-MIRIM,RO,1100106
1,2,"8,4745E+13",ALTO ALEGRE DOS PARECIS,RO,1100379
2,3,"5,90313E+12",PORTO VELHO,RO,1100205
3,8,"8,47369E+13",CUJUBIM,RO,1100940
4,13,"4,09271E+12",VILHENA,RO,1100304
...,...,...,...,...,...
5584,1190,"1,73498E+13",MOJUI DOS CAMPOS,PA,1504752
5585,1192,"1,72431E+13",BALNEARIO RINCAO,SC,4220000
5586,1194,"1,67808E+13",PESCARIA BRAVA,SC,4212650
5587,1196,"1,73616E+13",PARAISO DAS AGUAS,MS,5006275


In [ ]:
codigo_ibge_ajustado = municipios_siafi['codigo_ibge'].map(lambda x: str(x)[:-1])

In [ ]:
municipios_siafi = pd.concat([municipios_siafi, codigo_ibge_ajustado], axis=1)

In [ ]:
municipios_siafi['codigo_municipio_siafi'] = municipios_siafi['codigo_municipio_siafi'].astype(int)
cnpj_final['codigo_municipio_siafi'] = cnpj_final['codigo_municipio_siafi'].astype(int)

In [ ]:
municipios_siafi.columns = ['codigo_municipio_siafi', 'cnpj_base', 'descricao_municipio', 'uf','codigo_ibge', 'codigo_ibge_ajustado']

In [ ]:
municipios_siafi = municipios_siafi.drop(['cnpj_base','uf'], axis=1) #eliminação das colunas desnecessárias

In [ ]:
cnpj_final = pd.merge(cnpj_final, municipios_siafi, how='left', on='codigo_municipio_siafi')

In [ ]:
cnpj_final

,cnpj_basico,cnpj_ordem,cnpj_dv,matriz_filial,codigo_situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,data_inicio_atividade,cnae_principal,codigo_municipio_siafi,capital_social,descricao_municipio,codigo_ibge,codigo_ibge_ajustado
0,29880642,0001,04,1,08,20190921,01,20180308,8712300,7107,"2000,00",SAO PAULO,3550308,355030
1,29880783,0001,27,1,08,20200106,01,20180308,8630503,4205,"95400,00",CALDAS,3110301,311030
2,29880797,0001,40,1,08,20200507,01,20180308,9511800,5403,"3500,00",UBERLANDIA,3170206,317020
3,29880805,0001,59,1,08,20190213,01,20180308,5620104,6623,"10000,00",JUQUIA,3526100,352610
4,29880816,0001,39,1,08,20190808,01,20180308,9602501,3105,"1000,00",ARACAJU,2800308,280030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157252,29880238,0001,30,1,08,20190401,01,20180308,4772500,8047,"40000,00",BLUMENAU,4202404,420240
11157253,29880354,0001,50,1,02,20210527,00,20180308,4712100,145,"5000,00",SENA MADUREIRA,1200500,120050
11157254,29880380,0001,88,1,08,20190516,01,20180308,7319002,7107,"1000,00",SAO PAULO,3550308,355030
11157255,26520602,0001,63,1,08,20191219,01,20161110,4789004,7107,"1000,00",SAO PAULO,3550308,355030


In [ ]:
covid_brasil_1 = pd.read_csv('//content/drive/MyDrive/TCC - PUC/HIST_PAINEL_COVIDBR_2020_Parte1_31mai2021.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0)
covid_brasil_2 = pd.read_csv('//content/drive/MyDrive/TCC - PUC/HIST_PAINEL_COVIDBR_2020_Parte2_31mai2021.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0)
covid_brasil_3 = pd.read_csv('//content/drive/MyDrive/TCC - PUC/HIST_PAINEL_COVIDBR_Parte3_31mai2021.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0)

In [ ]:
covid_brasil_todas = pd.concat([covid_brasil_1, covid_brasil_2, covid_brasil_3], axis=0)

In [ ]:
covid_brasil_todas

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125,0,0,0,0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125,1,1,0,0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125,1,0,0,0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125,1,0,0,0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125,2,1,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848465,Centro-Oeste,DF,BrasÃ­lia,53,530010,53001,DISTRITO FEDERAL,2021-05-27,21,3015268,402182,1205,8569,32,NaN,NaN,1
848466,Centro-Oeste,DF,BrasÃ­lia,53,530010,53001,DISTRITO FEDERAL,2021-05-28,21,3015268,403175,993,8600,31,NaN,NaN,1
848467,Centro-Oeste,DF,BrasÃ­lia,53,530010,53001,DISTRITO FEDERAL,2021-05-29,21,3015268,404190,1015,8621,21,NaN,NaN,1
848468,Centro-Oeste,DF,BrasÃ­lia,53,530010,53001,DISTRITO FEDERAL,2021-05-30,22,3015268,405001,811,8642,21,NaN,NaN,1


In [ ]:
covid_brasil_todas = covid_brasil_todas[covid_brasil_todas['codmun'] .notnull()] 
covid_brasil_todas = covid_brasil_todas[covid_brasil_todas['municipio'] .notnull()] 

In [ ]:
covid_brasil_todas = covid_brasil_todas.drop(['municipio','coduf','codRegiaoSaude','nomeRegiaoSaude','semanaEpi','Recuperadosnovos','emAcompanhamentoNovos','emAcompanhamentoNovos', 'interior/metropolitana'], axis=1) #eliminação das colunas desnecessárias

In [ ]:
covid_brasil_todas.columns = ['regiao', 'estado', 'codigo_ibge_ajustado', 'data_covid','populacaoTCU2019', 'casosAcumulado','casosNovos','obitosAcumulado','obitosNovos']

In [ ]:
data_removido = covid_brasil_todas['data_covid'].str.replace(r'\D', '')

In [ ]:
codigo_ibge_data = data_removido + covid_brasil_todas['codigo_ibge_ajustado']

In [ ]:
covid_brasil_todas['populacaoTCU2019'] = covid_brasil_todas['populacaoTCU2019'].astype(int)
covid_brasil_todas['casosAcumulado'] = covid_brasil_todas['casosAcumulado'].astype(int)
covid_brasil_todas['obitosAcumulado'] = covid_brasil_todas['obitosAcumulado'].astype(int)

In [ ]:
casos_acumulados_porcentagem = round(((covid_brasil_todas['casosAcumulado'] * 100) / (covid_brasil_todas['populacaoTCU2019'])),3)
obitos_acumulados_porcentagem = round(((covid_brasil_todas['obitosAcumulado'] * 100) / (covid_brasil_todas['populacaoTCU2019'])),3)

In [ ]:
covid_brasil_todas = pd.concat([covid_brasil_todas, casos_acumulados_porcentagem, obitos_acumulados_porcentagem, codigo_ibge_data], axis=1)

In [ ]:
covid_brasil_todas.columns = ['regiao', 'estado', 'codigo_ibge_ajustado','data_covid', 'populacaoTCU2019','casosAcumulado', 'casosNovos','obitosAcumulado','obitosNovos', 'casos_acumulados_porcentagem','obitos_acumulados_porcentagem','codigo_ibge_data']

In [ ]:
codigo_ibge_data_regiao = pd.DataFrame ("20210531" + cnpj_final['codigo_ibge_ajustado'])

In [ ]:
codigo_ibge_data_regiao.columns = ['codigo_ibge_data']

In [ ]:
codigo_ibge_data_regiao  = pd.merge(codigo_ibge_data_regiao, covid_brasil_todas, how='left', on='codigo_ibge_data')

In [ ]:
codigo_ibge_data_regiao = codigo_ibge_data_regiao.drop(['codigo_ibge_data','codigo_ibge_ajustado','data_covid','casosAcumulado','casosNovos','obitosAcumulado','obitosNovos','casos_acumulados_porcentagem','obitos_acumulados_porcentagem'], axis=1) #eliminação das colunas desnecessárias

In [ ]:
cnpj_final = pd.concat([cnpj_final, codigo_ibge_data_regiao], axis=1)

In [ ]:
codigo_ibge_data_covid = pd.DataFrame (np.where((cnpj_final['codigo_situacao_cadastral']=='08'), cnpj_final['data_situacao_cadastral'] + cnpj_final['codigo_ibge_ajustado'] , "20210531" + cnpj_final['codigo_ibge_ajustado']))

In [ ]:
cnpj_final = pd.concat([cnpj_final, codigo_ibge_data_covid], axis=1)

In [ ]:
cnpj_final.columns = ['cnpj_basico','cnpj_ordem','cnpj_dv','matriz_filial','codigo_situacao_cadastral','data_situacao_cadastral','motivo_situacao_cadastral','data_inicio_atividade','cnae_principal','codigo_municipio_siafi','capital_social','descricao_municipio','codigo_ibge','codigo_ibge_ajustado','regiao','estado','populacaoTCU2019','codigo_ibge_data']

In [ ]:
codigo_ibge_data_covid.columns = ['codigo_ibge_data']

In [ ]:
codigo_ibge_data_covid  = pd.merge(codigo_ibge_data_covid, covid_brasil_todas, how='left', on='codigo_ibge_data')

In [ ]:
codigo_ibge_data_covid = codigo_ibge_data_covid.drop(['codigo_ibge_data','regiao','estado','codigo_ibge_ajustado','data_covid','populacaoTCU2019','casosNovos','obitosNovos'], axis=1) #eliminação das colunas desnecessárias

In [ ]:
codigo_ibge_data_covid.columns = ['casosAcumulado_covid','obitosAcumulado_covid','casos_acumulados_porcentagem_covid','obitos_acumulados_porcentagem_covid']

In [ ]:
cnpj_final = pd.concat([cnpj_final, codigo_ibge_data_covid], axis=1)

In [ ]:
idh_municipios = pd.read_csv('//content/drive/MyDrive//TCC - PUC/IDH por Município - FINAL COM CÓDIGO IBGE.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0,decimal=',')

In [ ]:
idh_municipios

,Territorialidade,CODIGO IBGE,Posicao IDHM,IDHM,Posicao IDHM Educacao,IDHM Educacao,Posicao IDHM Longevidade,IDHM Longevidade,Posicao IDHM Renda,IDHM Renda
0,ABADIA DE GOIAS,5200050,117,"0,708",155,"0,622",151,"0,687",65,"0,83"
1,ABADIA DOS DOURADOS,3100104,136,"0,689",214,"0,563",145,"0,693",56,"0,839"
2,ABADIANIA,5200100,136,"0,689",198,"0,579",167,"0,671",54,"0,841"
3,ABAETE,3100203,127,"0,698",221,"0,556",118,"0,72",47,"0,848"
4,ABAETETUBA,1500107,197,"0,628",240,"0,537",259,"0,579",97,"0,798"
...,...,...,...,...,...,...,...,...,...,...
5560,XIQUE-XIQUE,2933604,240,"0,585",298,"0,479",275,"0,563",154,"0,741"
5561,ZABELE,2517407,202,"0,623",190,"0,587",271,"0,567",170,"0,725"
5562,ZACARIAS,3557154,96,"0,729",103,"0,674",143,"0,695",69,"0,826"
5563,ZE DOCA,2114007,230,"0,595",272,"0,505",279,"0,559",150,"0,745"


In [ ]:
idh_municipios = idh_municipios.drop(['Territorialidade','Posicao IDHM','Posicao IDHM Educacao','Posicao IDHM Longevidade','Posicao IDHM Renda'], axis=1) #eliminação das colunas desnecessárias

In [ ]:
idh_municipios.columns = ['codigo_ibge', 'IDH DO MUNICIPIO', 'IDHM EDUCACAO DO MUNICIPIO', 'IDHM LONGEVIDADE DO MUNICIPIO','IDHM RENDA DO MUNICIPIO']

In [ ]:
cnpj_final = pd.merge(cnpj_final, idh_municipios, how='left', on='codigo_ibge')

In [ ]:
pib_municipios = pd.read_csv('//content/drive/MyDrive/TCC - PUC/PIB dos Municípios - base de dados 2010-2018 - AJUSTADO.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0,decimal=',')

In [ ]:
pib_municipios

,Sigla da Unidade da Federação,Código do Município,Nome do Município,PIB,PIB PER CAPITA,Ano
0,AC,1200013,Acrelândia,"236.379.386,00","15.737,64",2018
1,AC,1200054,Assis Brasil,"91.485.893,00","12.532,31",2018
2,AC,1200104,Brasiléia,"417.839.292,00","16.165,25",2018
3,AC,1200138,Bujari,"157.797.041,00","15.606,47",2018
4,AC,1200179,Capixaba,"170.402.523,00","14.874,52",2018
...,...,...,...,...,...,...
5565,TO,1721208,Tocantinópolis,"295.668.770,00","12.913,56",2018
5566,TO,1721257,Tupirama,"49.161.629,00","26.430,98",2018
5567,TO,1721307,Tupiratins,"29.765.709,00","11.391,39",2018
5568,TO,1722081,Wanderlândia,"140.613.090,00","12.089,51",2018


In [ ]:
pib_municipios = pib_municipios.drop(['Sigla da Unidade da Federação','Nome do Município','Ano'], axis=1) #eliminação das colunas desnecessárias

In [ ]:
pib_municipios.columns = ['codigo_ibge', 'PIB', 'PIB PER CAPITA']

In [ ]:
cnpj_final = pd.merge(cnpj_final, pib_municipios, how='left', on='codigo_ibge')

In [ ]:
cnae = pd.read_csv('//content/drive/MyDrive/TCC - PUC/CLASSIFICACAO CNAE POR SECAO.csv',encoding='ISO-8859-1', sep=';' ,dtype=str, header=0)

In [ ]:
cnae.columns = ['cnae_divisao', 'cnae_secao','cnae_descricao']

In [ ]:
cnae

,cnae_divisao,cnae_secao,cnae_descricao
0,1,a,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES..."
1,2,a,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES..."
2,3,a,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES..."
3,5,b,INDÚSTRIAS EXTRATIVAS
4,6,b,INDÚSTRIAS EXTRATIVAS
...,...,...,...
82,94,s,OUTRAS ATIVIDADES DE SERVIÇOS
83,95,s,OUTRAS ATIVIDADES DE SERVIÇOS
84,96,s,OUTRAS ATIVIDADES DE SERVIÇOS
85,97,t,SERVIÇOS DOMÉSTICOS


In [ ]:
cnae_divisao = cnpj_final['cnae_principal'].map(lambda x: str(x)[:-5])

In [ ]:
cnae_divisao = cnae_divisao.astype(int)

In [ ]:
cnae['cnae_divisao'] = cnae['cnae_divisao'].astype(int)

In [ ]:
secao_condicao = [
    (cnae['cnae_secao'] == 'a'),
    (cnae['cnae_secao'] == 'b'),
    (cnae['cnae_secao'] == 'c'),
    (cnae['cnae_secao'] == 'd'),
    (cnae['cnae_secao'] == 'e'),
    (cnae['cnae_secao'] == 'f'),
    (cnae['cnae_secao'] == 'g'),
    (cnae['cnae_secao'] == 'h'),
    (cnae['cnae_secao'] == 'i'),
    (cnae['cnae_secao'] == 'j'),
    (cnae['cnae_secao'] == 'k'),
    (cnae['cnae_secao'] == 'l'),
    (cnae['cnae_secao'] == 'm'),
    (cnae['cnae_secao'] == 'n'),
    (cnae['cnae_secao'] == 'o'),
    (cnae['cnae_secao'] == 'p'),
    (cnae['cnae_secao'] == 'q'),
    (cnae['cnae_secao'] == 'r'),
    (cnae['cnae_secao'] == 's'),
    (cnae['cnae_secao'] == 't'),
    (cnae['cnae_secao'] == 'u')]
cnae_secao = ['1', '2', '3','4', '5', '6','7', '8', '9','10', '11', '12','13', '14', '15','16', '17', '18','19', '20', '21']
cnae['cnae_secao_numero'] = np.select(secao_condicao, cnae_secao)

In [ ]:
cnpj_final = pd.concat([cnpj_final, cnae_divisao], axis=1)

In [ ]:
cnpj_final.columns = ['cnpj_basico','cnpj_ordem','cnpj_dv','matriz_filial','codigo_situacao_cadastral','data_situacao_cadastral','motivo_situacao_cadastral','data_inicio_atividade','cnae_principal','codigo_municipio_siafi','capital_social','descricao_municipio','codigo_ibge','codigo_ibge_ajustado','regiao','estado','populacaoTCU2019','codigo_ibge_data','casosAcumulado_covid','obitosAcumulado_covid','casos_acumulados_porcentagem_covid','obitos_acumulados_porcentagem_covid','IDH DO MUNICIPIO','IDHM EDUCACAO DO MUNICIPIO','IDHM LONGEVIDADE DO MUNICIPIO','IDHM RENDA DO MUNICIPIO','PIB','PIB PER CAPITA','cnae_divisao']

In [ ]:
cnpj_final = pd.merge(cnpj_final, cnae, how='left', on='cnae_divisao')

In [ ]:
cnpj_final['PIB'] = cnpj_final['PIB'].str.replace('.', '')
cnpj_final['PIB PER CAPITA'] = cnpj_final['PIB PER CAPITA'].str.replace('.', '')

In [ ]:
cnpj_final['capital_social'] = cnpj_final['capital_social'].str.replace(',', '.')
cnpj_final['IDH DO MUNICIPIO'] = cnpj_final['IDH DO MUNICIPIO'].str.replace(',', '.')
cnpj_final['IDHM EDUCACAO DO MUNICIPIO'] = cnpj_final['IDHM EDUCACAO DO MUNICIPIO'].str.replace(',', '.')
cnpj_final['IDHM LONGEVIDADE DO MUNICIPIO'] = cnpj_final['IDHM LONGEVIDADE DO MUNICIPIO'].str.replace(',', '.')
cnpj_final['IDHM RENDA DO MUNICIPIO'] = cnpj_final['IDHM RENDA DO MUNICIPIO'].str.replace(',', '.')
cnpj_final['PIB'] = cnpj_final['PIB'].str.replace(',', '.')
cnpj_final['PIB PER CAPITA'] = cnpj_final['PIB PER CAPITA'].str.replace(',', '.')

In [ ]:
cnpj_final = cnpj_final.drop(['codigo_ibge_ajustado','codigo_ibge_data'], axis=1) #eliminação das colunas desnecessárias

In [ ]:
data = {
'codigo_situacao_cadastral': ['02', '08'],
'situacao_cadastral_modelo': ['0', '1']
}

In [ ]:
situacao_cadastral = pd.DataFrame(data, columns=['codigo_situacao_cadastral','situacao_cadastral_modelo'])

In [ ]:
cnpj_final = pd.merge(cnpj_final, situacao_cadastral, how='left', on='codigo_situacao_cadastral')

In [ ]:
cnpj_final.to_csv('/content/drive/MyDrive/TCC - PUC/dataset_final.csv', index=False, sep=';', encoding='ISO-8859-1')

In [ ]:
cnpj_final

,cnpj_basico,cnpj_ordem,cnpj_dv,matriz_filial,codigo_situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,data_inicio_atividade,cnae_principal,codigo_municipio_siafi,capital_social,descricao_municipio,codigo_ibge,regiao,estado,populacaoTCU2019,casosAcumulado_covid,obitosAcumulado_covid,casos_acumulados_porcentagem_covid,obitos_acumulados_porcentagem_covid,IDH DO MUNICIPIO,IDHM EDUCACAO DO MUNICIPIO,IDHM LONGEVIDADE DO MUNICIPIO,IDHM RENDA DO MUNICIPIO,PIB,PIB PER CAPITA,cnae_divisao,cnae_secao,cnae_descricao,cnae_secao_numero,situacao_cadastral_modelo
0,29880642,0001,04,1,08,20190921,01,20180308,8712300,7107,2000.00,SAO PAULO,3550308,Sudeste,SP,12252023.0,NaN,NaN,NaN,NaN,0.805,0.725,0.843,0.855,714683362463.00,58691.90,87,q,SAÚDE HUMANA E SERVIÇOS SOCIAIS,17,1
1,29880783,0001,27,1,08,20200106,01,20180308,8630503,4205,95400.00,CALDAS,3110301,Sudeste,MG,14480.0,NaN,NaN,NaN,NaN,0.687,0.529,0.701,0.873,236518465.00,16405.53,86,q,SAÚDE HUMANA E SERVIÇOS SOCIAIS,17,1
2,29880797,0001,40,1,08,20200507,01,20180308,9511800,5403,3500.00,UBERLANDIA,3170206,Sudeste,MG,691305.0,199.0,9.0,0.029,0.001,0.789,0.716,0.776,0.885,37442790976.00,54801.25,95,s,OUTRAS ATIVIDADES DE SERVIÇOS,19,1
3,29880805,0001,59,1,08,20190213,01,20180308,5620104,6623,10000.00,JUQUIA,3526100,Sudeste,SP,18812.0,NaN,NaN,NaN,NaN,0.7,0.637,0.654,0.823,328898903.00,17394.70,56,i,ALOJAMENTO E ALIMENTAÇÃO,9,1
4,29880816,0001,39,1,08,20190808,01,20180308,9602501,3105,1000.00,ARACAJU,2800308,Nordeste,SE,657013.0,NaN,NaN,NaN,NaN,0.77,0.708,0.784,0.823,17276300787.00,26622.38,96,s,OUTRAS ATIVIDADES DE SERVIÇOS,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157252,29880238,0001,30,1,08,20190401,01,20180308,4772500,8047,40000.00,BLUMENAU,4202404,Sul,SC,357199.0,NaN,NaN,NaN,NaN,0.806,0.722,0.812,0.894,16958783360.00,48115.48,47,g,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,7,1
11157253,29880354,0001,50,1,02,20210527,00,20180308,4712100,145,5000.00,SENA MADUREIRA,1200500,Norte,AC,45848.0,5687.0,63.0,12.404,0.137,0.603,0.456,0.593,0.81,518416142.00,11475.22,47,g,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,7,0
11157254,29880380,0001,88,1,08,20190516,01,20180308,7319002,7107,1000.00,SAO PAULO,3550308,Sudeste,SP,12252023.0,NaN,NaN,NaN,NaN,0.805,0.725,0.843,0.855,714683362463.00,58691.90,73,m,"ATIVIDADES PROFISSIONAIS, CIENTÍFICAS E TÉCNICAS",13,1
11157255,26520602,0001,63,1,08,20191219,01,20161110,4789004,7107,1000.00,SAO PAULO,3550308,Sudeste,SP,12252023.0,NaN,NaN,NaN,NaN,0.805,0.725,0.843,0.855,714683362463.00,58691.90,47,g,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,7,1


In [ ]:
#tt = cnpj_final [0:50000]

In [ ]:
#tt.to_csv('/content/drive/MyDrive/TCC - PUC/dataset_final_exemplo.csv', index=False, sep=';', encoding='ISO-8859-1')